# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [109]:
def priest(sex, age, respiratory_rate, o2_sat, heart_rate, bp_sys, temp_C, alertness, o2_inspired, performance_status):
    '''The first part of this function calculates the severity of COVID-19 in a person based on their
    vitals and a few other parameters.The algorithm can be found on the MDCalc webpage.
    Then, based on the score, we can estimate a probability of adverse effects in the next 30 days.

    >>> priest('FEMALE', 40, 24, 0.96, 105, 115, 39.9, 'Alert', 'AIR', 'Unrestricted normal activity')
    0.09
    >>> priest('MaLe', 32, 20, 0.93, 100, 110, 32.2, 'CONFUSED OR NOT ALERT', 'SUPPLEMENTAL OXYGEN', 'LIMITED SELF-CARE')
    0.55
    >>> priest('FemalE', 22, 17, 0.98, 96, 103, 31.0, 'AleRT', 'AiR', 'BED/CHAIR bOuNd, no self-care')
    0.26
    >>> priest('MalE', 65, 24, 0.87, 90, 99, 36.5, 'ConFusEd oR nOt AleRt', 'air', 'limited strenuous activity, can do light activity')
    0.47
    >>> priest('male', 50, 20, 0.90, 87, 95, 35, 'alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.59
    '''
    if sex.lower() not in ['male', 'female']:
        raise ValueError(f'Invalid input for Sex: {sex.lower()}')

    if sex.lower() == "male":
        sex = 1
    elif sex.lower() == "female":
        sex = 0
    if age >= 16 and age <= 49:
        age = 0
    elif age >= 50 and age <= 65:
        age = 2
    elif age >= 66 and age <= 80:
        age = 3
    elif age > 80:
        age = 4

    if respiratory_rate < 9:
        respiratory_rate = 3
    elif respiratory_rate >= 9 and respiratory_rate <= 11:
        respiratory_rate = 1
    elif respiratory_rate >= 12 and respiratory_rate <= 20:
        respiratory_rate = 0
    elif respiratory_rate >= 21 and respiratory_rate <= 24:
        respiratory_rate = 2
    elif respiratory_rate > 24:
        respiratory_rate = 3

    if o2_sat > 0.95:
        o2_sat = 0
    elif o2_sat >= 0.94 and o2_sat <= 0.95:
        o2_sat = 1
    elif o2_sat >= 0.92 and o2_sat <= 0.93:
        o2_sat = 2
    elif o2_sat < 0.92:
        o2_sat = 3

    if heart_rate < 41:
        heart_rate = 3
    elif heart_rate >= 41 and heart_rate <= 50:
        heart_rate = 1
    elif heart_rate >= 51 and heart_rate <= 90:
        heart_rate = 0
    elif heart_rate >= 91 and heart_rate <= 110:
        heart_rate = 1
    elif heart_rate >= 111 and heart_rate <= 130:
        heart_rate = 2
    elif heart_rate >130:
        heart_rate = 3

    if bp_sys < 91:
        bp_sys = 3
    elif bp_sys >= 91 and bp_sys <= 100:
        bp_sys = 2
    elif bp_sys >= 101 and bp_sys <= 110:
        bp_sys = 1
    elif bp_sys >= 111 and bp_sys <= 219:
        bp_sys = 0
    elif bp_sys > 219:
        bp_sys = 3

    if temp_C < 35.1:
        temp_C = 3
    elif temp_C >= 35.1 and temp_C <= 36.0:
        temp_C = 1
    elif temp_C >= 36.1 and temp_C <= 38.0:
        temp_C = 0
    elif temp_C >= 38.1 and temp_C <= 39.0:
        temp_C = 1
    elif temp_C > 39.0:
        temp_C = 2

    if alertness.lower() == 'alert':
        alertness = 0
    elif alertness.lower() == 'confused or not alert':
        alertness = 3

    if o2_inspired.lower() == 'air':
        o2_inspired = 0
    elif o2_inspired.lower() == 'supplemental oxygen':
        o2_inspired = 2

    if performance_status.lower() == 'unrestricted normal activity':
        performance_status = 0
    elif performance_status.lower() == 'limited strenuous activity, can do light activity':
        performance_status = 1
    elif performance_status.lower() == 'limited activity, can self-care':
        performance_status = 2
    elif performance_status.lower() == 'limited self-care':
        performance_status = 3
    elif performance_status.lower() == 'bed/chair bound, no self-care':
        performance_status = 4
    risk_score = sum([sex, age, respiratory_rate, o2_sat, heart_rate, bp_sys, temp_C, alertness, o2_inspired, performance_status])

    if risk_score >= 0 and risk_score <= 1:
        risk = 0.01
    elif risk_score >= 2 and risk_score <= 3:
        risk = 0.02
    elif risk_score == 4:
        risk = 0.03
    elif risk_score == 5:
        risk = 0.09
    elif risk_score == 6:
        risk = 0.15
    elif risk_score == 7:
        risk = 0.18
    elif risk_score == 8:
        risk = 0.22
    elif risk_score == 9:
        risk = 0.26
    elif risk_score == 10:
        risk = 0.29
    elif risk_score == 11:
        risk = 0.34
    elif risk_score == 12:
        risk = 0.38
    elif risk_score == 13:
        risk = 0.46
    elif risk_score == 14:
        risk = 0.47
    elif risk_score == 15:
        risk = 0.49
    elif risk_score == 16:
        risk = 0.55
    elif risk_score == 17 and risk_score <= 25:
        risk = 0.59
    elif risk_score >= 26:
        risk = 0.99
    return risk

In [110]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('FEMALE', 40, 24, 0.96, 105, 115, 39.9, 'Alert', 'AIR', 'Unrestricted normal activity')
Expecting:
    0.09
ok
Trying:
    priest('MaLe', 32, 20, 0.93, 100, 110, 32.2, 'CONFUSED OR NOT ALERT', 'SUPPLEMENTAL OXYGEN', 'LIMITED SELF-CARE')
Expecting:
    0.55
ok
Trying:
    priest('FemalE', 22, 17, 0.98, 96, 103, 31.0, 'AleRT', 'AiR', 'BED/CHAIR bOuNd, no self-care')
Expecting:
    0.26
ok
Trying:
    priest('MalE', 65, 24, 0.87, 90, 99, 36.5, 'ConFusEd oR nOt AleRt', 'air', 'limited strenuous activity, can do light activity')
Expecting:
    0.47
ok
Trying:
    priest('male', 50, 20, 0.90, 87, 95, 35, 'alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.59
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [111]:
import requests
import json
def find_hospital(age, sex, risk):
    '''This function employs the capabilities of the REST web API to find the nearest facility to take a patient in an
    emergency based on their age, gender, and risk score.

    >>> find_hospital(45, 'male', 0.2)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(65, 'male', 0.49)
    'Wesley Woods Geriatric Hospital'
    >>> find_hospital(29, 'female', 0.56)
    'Emory Dunwoody Medical Center'
    >>> find_hospital (19, 'female', 0.16)
    'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospital(22, 'female', 0.92)
    'Emory Dunwoody Medical Center'
    '''
    url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
    response = requests.get(url)
    hospital_name = response.json()['hospital']
    return (hospital_name)

In [112]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(45, 'male', 0.2)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(65, 'male', 0.49)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(29, 'female', 0.56)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital (19, 'female', 0.16)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(22, 'female', 0.92)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [113]:
import requests

def get_address(hospital_name):
    """
    Retrieve address of a hospital based on its name from a website containing a list of hospitals.

    Statements:
    hospital_name (str): The name of the hospital to search for.

    Returns:
    str: The address of the hospital if found, None if not found.

    Examples:
    >>> get_address('Select Speciality Hospital - Northeast Atlanta')
    '1821 CLIFTON ROAD NE'
    >>> get_address('Emory Dunwoody Medical Center')
    '4500 NORTH SHALLOWFORD ROAD'
    >>> get_address('Wesley Woods Geriatric Hospital')
    '1821 CLIFTON ROAD, NE'
    """
    try:
        response = requests.get('https://hds5210-data.s3.amazonaws.com/ga_hospitals.json')
        response.raise_for_status()
        data = response.text
        hospital_data = json.loads(data)
        hospital_name=hospital_name.upper()
        if hospital_name in hospital_data:
          specific_hospital = hospital_data[hospital_name]
          if specific_hospital['City'] == "ATLANTA":
            return specific_hospital["ADDRESS"]
        if hospital_name.find("NORTHEAST ATLANTA"):
          return hospital_data['SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA']['ADDRESS']

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

In [114]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('Select Speciality Hospital - Northeast Atlanta')
Expecting:
    '1821 CLIFTON ROAD NE'
ok
Trying:
    get_address('Emory Dunwoody Medical Center')
Expecting:
    '4500 NORTH SHALLOWFORD ROAD'
ok
Trying:
    get_address('Wesley Woods Geriatric Hospital')
Expecting:
    '1821 CLIFTON ROAD, NE'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [115]:
def process_people():
    results = {}
    file_name = 'people.psv'
    try:
        with open("/content/" + file_name) as CSV:

            people_data = CSV.readlines()
            for people in people_data[1:]:
                try:
                    patient_details = people.split("|")
                    patient_number = patient_details[0]

                    risk_score = priest(patient_details[1], int(patient_details[2]), int(patient_details[3]), float(patient_details[4]),
                                       int(patient_details[5]), int(patient_details[6]), float(patient_details[7]), patient_details[8],
                                       patient_details[9], (patient_details[10][0:-1]))

                    hospital_name = find_hospital(int(patient_details[2]), patient_details[1], risk_score)

                    hospital_address = get_address(hospital_name)

                    patient_data = [
                        patient_details[1],
                        int(patient_details[2]),
                        float(patient_details[3]),
                        float(patient_details[4]),
                        float(patient_details[5]),
                        float(patient_details[6]),
                        float(patient_details[7]),
                        patient_details[8],
                        patient_details[9],
                        (patient_details[10][0:-1]),
                        risk_score,
                        hospital_name,
                        hospital_address
                    ]
                    results[patient_number] = patient_data
                except Exception as e:
                    print(f"Error processing data: {e}")

        return results
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")

        return None





## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [116]:
import json
json_file = "people_results.json"
with open("/content/" +  json_file) as file:
  json_data = json.loads(file.read())
  people_results = process_people()
  if json_data == people_results:
    print("Both are Same")

Both are Same


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---